In this notebook the custom model is trained with the best parameters from the HPO.

In [1]:
#Tensorflow Import and GPU recognition
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8988885223254764391, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16305806687984993003
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3016545379236591008
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13506356169886679055
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!unzip '/content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip' -d '/content'

Archive:  /content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip
replace /content/fruits-360_dataset/fruits-360/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [5]:
# importing some useful libs
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob
import numpy as np

In [6]:
#Defining dataset path and saving it for future usage
path = '/content/fruits-360_dataset/fruits-360'
os.listdir(path)

['Training', 'LICENSE', 'Test', 'papers', 'readme.md', 'test-multiple_fruits']

In [7]:
#Number of pictures in Training folder
training_files = glob(os.path.join(path,'Training', '*/*.jpg'))
image_num = len(training_files)
print("Number of Images: ",image_num)

Number of Images:  60498


In [8]:
#Number of pictures in Test Folder
testing_files = glob(os.path.join(path, 'Test', '*/*.jpg'))
img_num = len(testing_files)
print("Number of Images: ", img_num)

Number of Images:  20622


In [9]:
#Print category details
image_count = []
class_names = []
print('{:18s}'.format('Class'), end='')
print('Count:')
print('-'*24)
for folder in os.listdir(os.path.join(path,'Training')):
  folder_count = len(os.listdir(os.path.join(path,'Training',folder)))
  image_count.append(folder_count)
  class_names.append(folder)
  print('{:20s}'.format(folder), end='')
  print(folder_count)
print('-'*24)
print('Number of Classes:', len(class_names))
print('Average number of images per Class: ', np.array(image_count).mean())

Class             Count:
------------------------
Pear Abate          490
Nectarine           492
Apple Golden 3      481
Pear Forelle        702
Tomato Maroon       367
Pepper Green        444
Tomato 4            479
Apple Red Yellow 2  672
Tangelo             490
Potato Red Washed   453
Physalis with Husk  492
Carambula           490
Walnut              735
Pepino              490
Pear Kaiser         300
Apple Red 1         492
Kumquats            490
Mulberry            492
Mango Red           426
Plum 3              900
Melon Piel de Sapo  738
Peach Flat          492
Peach 2             738
Cauliflower         702
Orange              479
Kohlrabi            471
Grapefruit White    492
Cherry Wax Yellow   492
Cantaloupe 2        492
Apple Golden 1      492
Apple Crimson Snow  444
Grape White         490
Pepper Red          666
Huckleberry         490
Potato Red          450
Pomelo Sweetie      450
Kaki                490
Nut Pecan           178
Tomato 2            672
Quince        

In [10]:
#Definitive paths
train_out_path = os.path.join(path,'Training')
test_out_path = os.path.join(path, 'Test')
print(train_out_path)
print(test_out_path)

/content/fruits-360_dataset/fruits-360/Training
/content/fruits-360_dataset/fruits-360/Test


In [11]:
#Implementing some augmentation to avoid overfitting on the training generator
train_datagenerator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, validation_split=0.2, data_format='channels_last')
train_and_val_generator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, data_format='channels_last')
test_datagenerator = ImageDataGenerator(rescale= 1./255, data_format='channels_last')

In [12]:
#Creating Batches
image_size = (256, 256)
train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed=20052020)
val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', shuffle=False, seed=20052020)
train_val_batches = train_and_val_generator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False, seed=20052020)
test_batches = test_datagenerator.flow_from_directory(directory=test_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False)

Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 60498 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [15]:
# imports for building the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalMaxPool2D, Dropout,  Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [16]:
#build custom model
model = Sequential()
model.add( Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same', data_format='channels_last', activation='relu', name='conv_1', input_shape=image_size+(3,)))
model.add( MaxPool2D(pool_size=(2, 2), strides=1, data_format='channels_last', name='pool_1'))
model.add( Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', data_format='channels_last', activation='relu', name='conv_2'))
model.add( GlobalMaxPool2D(data_format='channels_last', name='pool_2'))
model.add( Dropout(0.08832308429673111))
model.add( Dense(120, activation='softmax', name='prediction', kernel_regularizer=l2(0.09116292059447313)))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0029464233838192235), metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 256, 256, 128)     3584      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 255, 255, 128)     0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 255, 255, 64)      73792     
_________________________________________________________________
pool_2 (GlobalMaxPooling2D)  (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
prediction (Dense)           (None, 120)               7800      
Total params: 85,176
Trainable params: 85,176
Non-trainable params: 0
____________________________________________________

In [ ]:
'''
model_path = '/content/drive/My Drive/Progetto Advanced Machine Learning/Model Checkpoints/Custom Model 1 2020-06-01 10:56:21.010759.hdf5'
model = tf.keras.models.load_model(model_path)
x = model.layers[-2].output
x = Dropout(0.00032332824201601514)(x)
predictions = Dense(120, activation='softmax', name='prediction', kernel_regularizer=l2(0.24718165404756284))(x)
model = Model(inputs=model.inputs, outputs=predictions)
for layer in model.layers[:-2]:
  layer.trainable = False
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0980741228990046), metrics=['accuracy'])

model.summary()
'''

In [17]:
# training the model with early stopping
model.save_weights('Initial Weights.hdf5')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(train_batches, epochs=15, verbose=1, shuffle=True, validation_data=val_batches, callbacks=[early_stopping], workers=2)

Epoch 1/15
1514/1514 [==============================] - ETA: 0s - loss: 3.0970 - accuracy: 0.3237

KeyboardInterrupt: ignored

In [ ]:
model.load_weights('Initial Weights.hdf5')
trainingMdl = model.fit(train_val_batches, epochs=1early_stopping.stopped_epoch, verbose=1, shuffle=True, validation_data=test_batches, workers=2)

In [ ]:
#plot training history
import sys
sys.path.append('/content/drive/My Drive/Progetto Advanced Machine Learning/')
import utils
history_fig = utils.plot_history(trainingMdl)
history_fig.set_size
history_fig.show()

In [ ]:
import datetime
save_path = '/content/drive/My Drive/Progetto Advanced Machine Learning/Model Checkpoints/'
save_path += 'Custom Model 1 After HPO'
save_path += str(datetime.datetime.now())
model.save(save_path + '.hdf5')

In [ ]:
#save training history plot
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
history_fig.write_image(save_path +'.pdf', format='pdf')

In [ ]:
#save training history
import pickle
with open(save_path +'.history.pickle', 'wb') as f:
  pickle.dump(trainingMdl.history, f)

In [ ]:
cm = utils.get_confusion_matrix(model, val_batches)
fig = utils.plot_cm(cm, class_names)
plt.savefig(save_path + '.confusion_matrix.pdf', format='pdf')
fig